## First, loading the data and defining the functions

In [1]:
import pandas as pd
# df_letters = pd.read_csv('data/all_letters.tsv',sep='\t').drop('Unnamed: 0',axis=1) # remove "drop" if the column is not there
df_letters = pd.read_csv('data/letters_latest.tsv',sep='\t').drop('Unnamed: 0',axis=1) # remove "drop" if the column is not there

In [2]:
df_commodities = pd.read_csv('data/commodities_latest.tsv',sep='\t').drop('Unnamed: 0',axis=1)

In [42]:
# manually adding historical spelling variations for spices we're interested in
# from https://gtb.ivdnt.org/search/

historical_labels = {'Kaneel': ['canella','caneel','canele','caneele',
                               'kanele','canelle'],
                    'Nootmuskaat':['notemosscate', 'notemetscate', 'notemisscate','musschaeten',
                                  'notemusscate','note musscate','muscaetnot','muskaatnoot',
                                  'muskaat','musscat','muscaet','metscat','misscat','mosscat'],
                    'Foelie':['folye','folie']}

for main_label, extra_labels in historical_labels.items():
    df_commodities.loc[df_commodities['main_label']==main_label,'all_labels']+=';'+';'.join(extra_labels)

In [43]:
commodities = df_commodities.to_dict(orient='records')

In [44]:
import json

In [45]:
global matching_mode
'''
options:
        "exact": "kaneel" and "kaneelstokjes" won't match (minimise false positives)
        "broad": "gin" and "beginnen" will match (minimise false negatives)
        "left_only": "kaneel" and "kaneelstokjes" match but "gin" and "beginnen" don't (a middle ground)
'''


'\noptions:\n        "exact": "kaneel" and "kaneelstokjes" won\'t match (minimise false positives)\n        "broad": "gin" and "beginnen" will match (minimise false negatives)\n        "left_only": "kaneel" and "kaneelstokjes" match but "gin" and "beginnen" don\'t (a middle ground)\n'

In [46]:
import re

def find_label(text, label):
    if matching_mode == 'left_only':
        pattern = r'(?<![a-zA-Z])' + re.escape(label)
    if matching_mode == 'exact':
        pattern = r'(?<![a-zA-Z])' + re.escape(label) + r'(?![a-zA-Z])'
    if matching_mode == 'broad':
        pattern = re.escape(label)
        
    regex = re.compile(pattern)
    
    return re.search(regex, text)

In [47]:
matching_mode = 'exact'
find_label('We beginnen met een beetje gin', 'gin')

<re.Match object; span=(27, 30), match='gin'>

In [48]:
def find_commodities(text): # looking for all commodities, use this if you need to analyse other things
    res = {}
    if type(text) != str:
        return '{}'
    
    for row in commodities:
        concept_uri, labels = row['uri'], row['all_labels']
        cur_labels = set(labels.split(';'))
        for label in cur_labels:
            if find_label(text.lower(), label.lower()): 
                if concept_uri in res: # different labels for the same uri already found
                    res[concept_uri].append(label)
                else:
                    res[concept_uri] = [label]

    return json.dumps(res)

In [49]:
def find_opium_and_spices(text): # only looking for our target commodities
    res = {}
    if type(text) != str:
        return '{}'
    
    spicy_labels = ['opium', 'kaneel','nootmuskaat','foelie', 'peper']
    
    for row in commodities:
        concept_uri, labels = row['uri'], row['all_labels']
        cur_labels = labels.split(';')
        if cur_labels[0].lower() not in spicy_labels: # main label
            continue
        for label in set(cur_labels):
            if find_label(text.lower(), label.lower()): 
                if concept_uri in res: # different labels for the same uri already found
                    res[concept_uri].append(label)
                else:
                    res[concept_uri] = [label]

    return json.dumps(res)

## Quick solution starts here: only looking for opium and target spices

In [50]:
import mapply 
mapply.init()

matching_mode = 'left_only'
df_letters['commodities'] = df_letters['text'].mapply(find_opium_and_spices)
matching_mode = 'exact'
df_letters['commodities_precise'] = df_letters['text'].mapply(find_opium_and_spices)
df_letters.to_csv('data/letters_with_opium_and_spices.tsv',sep='\t',index=False)

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

## Extended solution starts here: if you want to analyse co-occurrences of all other commodities

In [51]:
% # If you have time, remove the '%' sign here, run this cell and ignore all the code below it:
import mapply 
mapply.init()

matching_mode = 'left_only'
df_letters['commodities'] = df_letters['text'].mapply(find_commodities)
matching_mode = 'exact'
df_letters['commodities_precise'] = df_letters['text'].mapply(find_commodities)
df_letters.to_csv('data/letters_with_commodities_latest.tsv',sep='\t',index=False)

UsageError: Line magic function `%` not found.


In [ ]:
### Quick and less readable solution starts here:

In [ ]:
df_letters['commodities'] = [{}]*len(df_letters)

In [ ]:
# df_letters = pd.read_csv('data/letters_with_commodities_latest.tsv',sep='\t') # if the connection was breaking

In [ ]:
import mapply # we need to speed things up because the GMs are super long and the deadline is soon
mapply.init()

In [23]:
matching_mode = 'left_only'
cur_start = 0
cur_end = 16
while cur_end < len(df_letters): # working in batches, doing things in parallel
    print(cur_start, cur_end) 
#     if df_letters.iloc[cur_start]['commodities'] != '{}' or df_letters.iloc[cur_end]['commodities'] != '{}':
#         cur_start = cur_end 
#         cur_end += 8
#         continue # if the connection was breaking and you have some results saved already
    df_letters['commodities'][cur_start:cur_end] = df_letters['text'][cur_start:cur_end].mapply(find_commodities)
    cur_start = cur_end 
    cur_end += 16
    
df_letters['commodities'][cur_start:len(df_letters)] = df_letters['text'][cur_start:len(df_letters)].mapply(find_commodities)

0 16


  0%|          | 0/1 [00:00<?, ?it/s]

16 32


/home/vprovat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

896 912


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

/home/vprovat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
df_letters.to_csv('data/letters_with_commodities_latest.tsv',sep='\t',index=False)

In [ ]:
df_letters['commodities_precise'] = [{}]*len(df_letters)

In [ ]:
matching_mode = 'exact'

cur_start = 0
cur_end = 32
while cur_end < len(df_letters): # working in batches, doing things in parallel
    print(cur_start, cur_end) 
#     if df_letters.iloc[cur_end]['commodities_precise'] != '{}' or df_letters.iloc[cur_start]['commodities_precise'] != '{}':
#         cur_start = cur_end 
#         cur_end += 8
#         continue # if the connection was breaking and you have some results saved already
    df_letters['commodities_precise'][cur_start:cur_end] = df_letters['text'][cur_start:cur_end].mapply(find_commodities)
    cur_start = cur_end 
    cur_end += 32
    
df_letters['commodities_precise'][cur_start:len(df_letters)] = df_letters['text'][cur_start:len(df_letters)].mapply(find_commodities)

In [ ]:
df_letters.to_csv('data/letters_with_commodities_latest.tsv',sep='\t',index=False)